In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
# connecting to the driver
driver = webdriver.Chrome(r"C:\Users\LENOVO\Downloads\chromedriver.exe")

### Q1. 
Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [7]:
# Take user input for product search
search_query = input("Enter the product name: ")

# Navigate to the Amazon India home page
driver.get("https://www.amazon.in/")

Enter the product name: guitar


In [8]:
# Find the search box and enter the search query
search_box = driver.find_element(By.ID, "twotabsearchtextbox")
search_box.send_keys(search_query)

go = driver.find_element(By.ID, "nav-search-submit-button")
go.click()
time.sleep(5)

### Q2.
Scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [9]:
# creating empty lists for data
brand = []
name = []
price = []
exchange = []
delivery = []
availability = []
url = []


In [10]:
# scraping products url from multiple pages
products_url = []
start = 0
end = 3
for page in range(start,end):
    url = driver.find_elements(By.XPATH, '//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
    
    for i in url:
        products_url.append(i.get_attribute('href'))
    try:
        next_button = driver.find_element(By.XPATH, '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
        next_button.click()
    except NoSuchElementException:
        next_button = driver.find_element(By.CSS_SELECTOR,"s-pagination-item s-pagination-next s-pagination-button s-pagination-separator")
        next_button.click()
    time.sleep(3)
    

In [11]:
len(products_url)

156

In [12]:
# data from every product url
for i in products_url:
    driver.get(i)
    time.sleep(5)
    
    try:
        br = driver.find_element(By.XPATH,'//*[@id="productOverview_feature_div"]/div/table/tbody/tr[1]/td[2]/span')
        brand.append(br.text)
    except NoSuchElementException:
        brand.append('-')
        
    try:
        pr = driver.find_element(By.XPATH,'//span[@class="a-price aok-align-center reinventPricePriceToPayMargin priceToPay"]')
        price.append(pr.text)
    except NoSuchElementException:
        price.append('-')
        
    try:
        ex = driver.find_element(By.XPATH,'//div[@class="a-carousel-viewport"]/ol/li[3]/div/span/div[2]/span' )
        exchange.append(ex.text)
    except NoSuchElementException:
        exchange.append('-')
        
    try:
        brand_name = driver.find_element(By.XPATH, '//span[@class="a-size-medium product-title-word-break product-title-resize"]')
        name.append(brand_name.text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        deli = driver.find_element(By.XPATH,'//div[@class="a-spacing-base"]/span/span[1]')
        delivery.append(deli.text)
    except NoSuchElementException:
        delivery.append('-')
        
    try:
        avail = driver.find_element(By.XPATH, '//span[@class="a-size-medium a-color-success"]')
        availability.append(avail.text)
    except NoSuchElementException:
        availability.append('-')
        

In [13]:
print(len(brand),len(name),len(price),len(availability),len(delivery),len(exchange))

156 156 156 156 156 156


In [15]:
#converting to a dataframe
guitar_data = pd.DataFrame({'Brand_Name':brand, 'Product_Name':name, 'Price':price, 'Delivery':delivery, 'Returns/Exchange':exchange, 'Availability':availability,'URL':products_url})
guitar_data

,Brand_Name,Product_Name,Price,Delivery,Returns/Exchange,Availability,URL
0,VAULT,-,"₹4,084","Sunday, 21 May",-,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Guitar Bro,-,"₹8,999",May 24 - 27,-,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Fender,-,"₹11,990",June 2 - 5,-,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Multicolor,-,"₹1,999","Tuesday, 23 May",-,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,Intern,-,"₹2,199","Tuesday, 23 May",-,-,https://www.amazon.in/Intern-INT-38C-Acoustic-...
...,...,...,...,...,...,...,...
151,YAMAHA,-,"₹9,930","Tuesday, 23 May",-,-,https://www.amazon.in/Yamaha-Folk-Guitar-Natur...
152,Medellin,-,"₹2,349","Tuesday, 23 May",-,In stock,https://www.amazon.in/Medellin-carbn-fiber-gui...
153,Kadence,-,"₹2,499","Tuesday, 23 May",-,In stock,https://www.amazon.in/Medellin-Acoustic-adjust...
154,Blueberry,-,"₹4,270","Sunday, 21 May",-,In stock,https://www.amazon.in/Blueberry-Acoustic-Inbui...


In [16]:
# dataframe to csv conversion
guitar_data.to_csv('guitar_data.csv', index=False)

### Q4.
Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [17]:
# connecting to the driver
driver = webdriver.Chrome(r"C:\Users\LENOVO\Downloads\chromedriver.exe")

In [18]:
# opening the website
driver.get("https://www.flipkart.com")
time.sleep(3)

In [21]:
try:
    cross = driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _2doB4z"]')
    cross.click()

    # Find the search box and enter the search query
    search_box = driver.find_element(By.CLASS_NAME, "_3704LK")
    search_box.send_keys('Smartphone')

    wait = WebDriverWait(driver, 10)
    go = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@class="L0Z3Pu"]')))
    go.click()
    
    time.sleep(5)

except NoSuchElementException:
    # Find the search box and enter the search query
    search_box = driver.find_element(By.CLASS_NAME, "_3704LK")
    search_box.send_keys('Smartphone')

    wait = WebDriverWait(driver, 10)
    go = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@class="L0Z3Pu"]')))
    go.click()
    
    time.sleep(5)
    

In [22]:
# scraping products url from the first page
products_url = []
url = driver.find_elements(By.XPATH, '//a[@class="_1fQZEK"]')    
for i in url:
    products_url.append(i.get_attribute('href'))
time.sleep(3)

In [23]:
len(products_url)

24

In [24]:
# creating empty lists for data
brand = []
name = []
color = [] 
RAM = []
storage = []
camera1 = []
camera2 = []
display = []
battery = []
price = []

In [25]:
for i in products_url:
    driver.get(i)
    time.sleep(5)
    
    try:
        info = driver.find_element(By.XPATH, '//span[@class="B_NuCI"]')
        name_text = info.text.split('(')[0]
        name.append(name_text)
    except NoSuchElementException:
        name.append('-')
        
    try:
        br = driver.find_element(By.XPATH, '//span[@class="B_NuCI"]')
        brand.append(br.text.split('(')[0].split()[0])
    except NoSuchElementException:
        brand.append('-')
        
    try:
        col = driver.find_element(By.XPATH, '//span[@class="B_NuCI"]')
        color_text = col.text.split('(')[1].split(',')[0]
        color.append(color_text)
    except NoSuchElementException:
        color.append('-')
      
    try:
        pr = driver.find_element(By.XPATH, '//div[@class="_30jeq3 _16Jk6d"]')
        price.append(pr.text)
    except NoSuchElementException:
        price.append('-')
        
    try:
        ram = driver.find_element(By.XPATH, '//li[@class="_21Ahn-"][1]')
        RAM.append(ram.text.split('|')[0])
    except NoSuchElementException:
        RAM.append('-')
        
    try:
        store = driver.find_element(By.XPATH, '//li[@class="_21Ahn-"][1]')
        storage.append(store.text.split('|')[1])
    except NoSuchElementException:
        storage.append('-')
        
    try:
        size = driver.find_element(By.XPATH, '//li[@class="_21Ahn-"][2]')
        display.append(size.text.split('(')[0].strip())
    except NoSuchElementException:
        display.append('-')
        
    try:
        primary = driver.find_element(By.XPATH, '//li[@class="_21Ahn-"][3]')
        camera1.append(primary.text.split('|')[1].split()[0])
    except NoSuchElementException:
        camera1.append('-')
        
    try:
        second = driver.find_element(By.XPATH, '//li[@class="_21Ahn-"][3]')
        camera2.append(second.text.split('|')[0])
    except NoSuchElementException:
        camera2.append('-')
        
    try:
        bat= driver.find_element(By.XPATH, '//li[@class="_21Ahn-"][4]')
        battery.append(bat.text.split()[0])
    except NoSuchElementException:
        battery.append('-')

In [26]:
print(len(brand), len(name), len(color), len(RAM), len(storage), len(camera1), len(camera2), len(display), len(battery),len(price))

24 24 24 24 24 24 24 24 24 24


In [27]:
# creating the dataframe
phone_data = pd.DataFrame({'Barnd':brand, 'Product_name':name, 'Price':price, 'Color':color, 'Display_size':display,
                          'Primary_camera':camera2, 'Secondary_camera':camera1, 'RAM':RAM, 'ROM/Storage':storage,
                          'Battery':battery, 'Product_URL':products_url})

In [28]:
# converting datfarame to csv
phone_data.to_csv('phone_data.csv', index = False)

In [29]:
phone_data

,Barnd,Product_name,Price,Color,Display_size,Primary_camera,Secondary_camera,RAM,ROM/Storage,Battery,Product_URL
0,REDMI,REDMI 10,"₹9,499",Caribbean Green,17.02 cm,50MP + 2MP,5MP,4 GB RAM,64 GB ROM,6000,https://www.flipkart.com/redmi-10-caribbean-gr...
1,realme,realme C33,"₹8,999",Night Sea,16.51 cm,50MP + 0.3MP,5MP,3 GB RAM,32 GB ROM,5000,https://www.flipkart.com/realme-c33-night-sea-...
2,MOTOROLA,MOTOROLA G32,"₹11,999",Satin Silver,16.64 cm,50MP + 8MP + 2MP,16MP,8 GB RAM,128 GB ROM,5000,https://www.flipkart.com/motorola-g32-satin-si...
3,realme,realme C33 2023,"₹9,999",Sandy Gold,16.51 cm,50MP + 0.3MP,5MP,4 GB RAM,64 GB ROM,5000,https://www.flipkart.com/realme-c33-2023-sandy...
4,realme,realme C33 2023,"₹9,999",Night Sea,16.51 cm,50MP + 0.3MP,5MP,4 GB RAM,64 GB ROM,5000,https://www.flipkart.com/realme-c33-2023-night...
5,realme,realme C33 2023,"₹9,999",Aqua Blue,16.51 cm,50MP + 0.3MP,5MP,4 GB RAM,64 GB ROM,5000,https://www.flipkart.com/realme-c33-2023-aqua-...
6,REDMI,REDMI 10,"₹9,499",Midnight Black,17.02 cm,50MP + 2MP,5MP,4 GB RAM,64 GB ROM,6000,https://www.flipkart.com/redmi-10-midnight-bla...
7,REDMI,REDMI 10,"₹9,499",Sunrise Orange,17.02 cm,50MP + 2MP,5MP,4 GB RAM,64 GB ROM,6000,https://www.flipkart.com/redmi-10-sunrise-oran...
8,POCO,POCO C51,"₹7,249",Royal Blue,16.56 cm,8MP Dual Rear Camera,5MP,4 GB RAM,64 GB ROM,5000,https://www.flipkart.com/poco-c51-royal-blue-6...
9,realme,realme C33,"₹8,999",Aqua Blue,16.51 cm,50MP + 0.3MP,5MP,3 GB RAM,32 GB ROM,5000,https://www.flipkart.com/realme-c33-aqua-blue-...


### Q5.
Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [32]:
# opening the website
driver.get("https://www.google.com/maps/")
time.sleep(5)

In [33]:
#asking the user to input city
city_name = str(input("Enter the city: "))

# finding the erach bar, entering the city 
search = driver.find_element(By.ID, 'searchboxinput')
search.send_keys(city_name)

go = driver.find_element(By.ID, 'searchbox-searchbutton')
go.click()
time.sleep(5)

Enter the city: ranchi


In [34]:
import re
# Get the current URL
try:
    url_string = driver.current_url
    print("URL Extracted: " + url_string)
    print('\n')
    lat_long = re.findall(r'@(.*)data',url_string)
    
    # city coordinates
    if lat_long:
        lat_long = lat_long[0].split(",")
        latitude = lat_long[0]
        longitude = lat_long[1]
        print("Latitude: " + latitude)
        print("Longitude: " + longitude)
    
except Exception as e:
    print(f"Error occurred: {e}")

URL Extracted: https://www.google.com/maps/place/Ranchi,+Jharkhand/@23.3434338,85.1565082,11z/data=!3m1!4b1!4m6!3m5!1s0x39f4e104aa5db7dd:0xdc09d49d6899f43e!8m2!3d23.3440997!4d85.309562!16zL20vMDF0dHQ2


Latitude: 23.3434338
Longitude: 85.1565082


### Q7.
Write a program to scrap all the available details of best gaming laptops from digit.in.

In [37]:
# opening the website
driver.get("https://www.digit.in/")

In [38]:
# finding the link of data
drop_down = driver.find_element(By.XPATH, '//span[@class="arrow_down more"]')
drop_down.click()

laptop = driver.find_element(By.XPATH, '//div[@class="mega_tabcontent more-1 active"]/div/ul[1]/li[2]/a')
laptop.click()

gaming_laptop = driver.find_element(By.XPATH, '//div[@class="listing_container"]/ul[1]/li[9]/a')
gaming_laptop.click()

In [39]:
# creating empty lists for data
laptop_name = []
price = []
processor =[]
display = []
os = []
memory = []
graphics = []
body = []

In [40]:
# scraping data
try:
    name = driver.find_elements(By.XPATH, '//span[@class="datahreflink"]/h3')
    for i in name:
        laptop_name.append(i.text)
except NoSuchElementException:
    laptop_name.append('-')
    
try:
    pr = driver.find_elements(By.XPATH, '//div[@class="Spcs-details"]/table/tbody/tr[8]/td[3]')
    for i in pr:
        price.append(i.text)
except NoSuchElementException:
    price.append('-')
    
try:
    pro = driver.find_elements(By.XPATH, '//div[@class="Spcs-details"]/table/tbody/tr[2]/td[3]')
    for i in pro:
        processor.append(i.text)
except NoSuchElementException:
    processor.append('-')
    
try:
    disp = driver.find_elements(By.XPATH, '//div[@class="Spcs-details"]/table/tbody/tr[3]/td[3]')
    for i in disp:
        display.append(i.text.split(',')[0])
except NoSuchElementException:
    display.append('-')
    
try:
    sys= driver.find_elements(By.XPATH, '//div[@class="Spcs-details"]/table/tbody/tr[4]/td[3]')
    for i in sys:
        os.append(i.text.split(',')[0])
except NoSuchElementException:
    os.append('-')
    
try:
    mm= driver.find_elements(By.XPATH, '//div[@class="Spcs-details"]/table/tbody/tr[5]/td[3]')
    for i in mm:
        memory.append(i.text)
except NoSuchElementException:
    memory.append('-')
    
try:
    graph= driver.find_elements(By.XPATH, '//div[@class="Spcs-details"]/table/tbody/tr[6]/td[3]')
    for i in graph:
        graphics.append(i.text)
except NoSuchElementException:
    graphics.append('-')
    
try:
    bd= driver.find_elements(By.XPATH, '//div[@class="Spcs-details"]/table/tbody/tr[7]/td[3]')
    for i in bd:
        body.append(i.text)
except NoSuchElementException:
    body.append('-')

In [41]:
# creating dataframe
laptop_data = pd.DataFrame({'Name':laptop_name, 'Price':price, 'Operating_sys':os, 'Display':display,'Memory':memory,
                            'Processor':processor, 'Body':body, 'Graphics_Processor':graphics})


In [42]:
laptop_data

,Name,Price,Operating_sys,Display,Memory,Processor,Body,Graphics_Processor
0,HP OMEN 17 (2023),"₹ 269,777",Windows 11 Home,17.3″ (2560 x 1440) screen,16 GB DDR5GB RAM & 1 TB SSD,13th Gen Intel Core i7-13700HX 16 core process...,397.1 x 262 x 27 mm dimension & 2.78 kg weight,12 GB DDR6 NVIDIA GeForce RTX 4080 Graphics card
1,MSI Titan GT77 12UHS,"₹ 499,990",Windows 11 Home,17.3″ (3840 x 2160) screen,64 GB DDR5GB RAM & 2 TB SSD,12th Gen Intel Core i9-12900HX 16 core process...,397 x 330 x 23 mm dimension & 3.3 kg weight,16 GB DDR6 NVIDIA GeForce RTX 3080Ti Graphics ...
2,Lenovo Legion 5i Pro,"₹ 218,000",Windows 11 Home,16″ (2560 x 1600) screen,32 GB DDR5GB RAM & 1 TB SSD,12th Gen Intel Core i7-12700H 14 core processo...,359.9 x 264.4 x 19.9 mm dimension & 3.6 kg weight,NVIDIA GeForce RTX 3070 Ti Graphics card
3,ASUS ROG Strix Scar 18 2023,"₹ 279,990",Windows 11 Home,18″ (1920 x 1200) screen,32 GB DDR5GB RAM & 1 TB SSD,13th Gen Intel Core i9-13980HX 24 core process...,294 x 399 x 23 mm dimension & 3.1 kg weight,12 GB DDR6 NVIDIA GeForce RTX 4080 Graphics card
4,Acer Predator Helios Neo 16,"₹ 149,990",Windows 11 Home,16″ (2560 x 1600) screen,16 GB DDR5GB RAM & 1 TB SSD,13th Gen Intel Core i7-13700HX processor,360 x 279 x 28 mm dimension & 2.6 kg weight,8 GB DDR6 NVIDIA GeForce RTX 4060 Graphics card
5,ASUS ROG Zephyrus G14,"₹ 156,990",Windows 11 Home,14″ (1920 x 1200) screen,16 GB DDR5GB RAM & 1 TB SSD,AMD Ryzen 9-6900HS 8 core processor with 4.9 G...,312 x 227 x 19 mm dimension & 1.65 kg weight,8 GB DDR6 AMD Radeon RX 6700S Graphics card
6,MSI Cyborg 15,"₹ 130,990",Windows 11 Home,15.6″ (1920 x 1080) screen,16 GB DDR5GB RAM & 1 TB SSD,12th Gen Intel Core i7-12650H 10 core processo...,& 1.98 kg weight,8 GB DDR6 NVIDIA GeForce RTX 4060 Graphics card


### Q8.
Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [5]:
# opening the website
driver.get("https://www.forbes.com/billionaires/")
time.sleep(10)

In [6]:
# finding the list
button = driver.find_element(By.XPATH, '//div[@class="_69hVhdY4"]')
button.click()

billion = driver.find_element(By.XPATH, '//div[@class="xMDGnUGv lXvj00YF pgYJ2orL"]/ul//li[2]/div/span')
billion.click()

people = driver.find_element(By.XPATH,'//div[@class="xMDGnUGv"]/ul/li[1]/a')
people.click()

In [7]:
# creating empty lists
name = []
rank = []
net_worth = []
age = []
citizenship = []
source = []
industry = []


In [9]:
# for scrapping data from multiple pages
start = 0
end = 14
for page in range(start,end):
    try:
        nm = driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[2]/div')
        for i in nm:
            name.append(i.text)
    except NoSuchElementException:
        name.append('-')
    
    try:
        rn = driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[1]/div')
        for i in rn:
            rank.append(i.text)
    except NoSuchElementException:
        rank.append('-')
    
    try:
        worth = driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[3]/div')
        for i in worth:
            net_worth.append(i.text)
    except NoSuchElementException:
        net_worth.append('-')
    
    try:
        ag = driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[4]/div')
        for i in ag:
            age.append(i.text)
    except NoSuchElementException:
        age.append('-')
    
    try:
        city= driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[5]')
        for i in city:
            citizenship.append(i.text)
    except NoSuchElementException:
        citizenship.append('-')
    
    try:
        sr= driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[6]/div/span')
        for i in sr:
            source.append(i.text)
    except NoSuchElementException:
        source.append('-')
    
    try:
        ind= driver.find_elements(By.XPATH,'//div[@class="TableRow_row__L-0Km"]/div[7]/div')
        for i in ind:
            industry.append(i.text)
    except NoSuchElementException:
        industry.append('-')
    
    # clicking on the next page
    next_page = driver.find_element(By.XPATH, '//div[@class="Pagination_pagination__V7MBu"]/button[2]/div')
    next_page.click()

ElementClickInterceptedException: Message: element click intercepted: Element <div class="Pagination_previousPage__0Eyky">...</div> is not clickable at point (320, 31). Other element would receive the click: <header class="egtB2NsG" role="banner">...</header>
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x01414AD3]
	(No symbol) [0x013A9211]
	(No symbol) [0x0129D9CD]
	(No symbol) [0x012D9072]
	(No symbol) [0x012D6730]
	(No symbol) [0x012D3BCB]
	(No symbol) [0x012D2308]
	(No symbol) [0x012C68D3]
	(No symbol) [0x012EFB3C]
	(No symbol) [0x012C61DF]
	(No symbol) [0x012EFE94]
	(No symbol) [0x013067B9]
	(No symbol) [0x012EF936]
	(No symbol) [0x012C4788]
	(No symbol) [0x012C5C1D]
	GetHandleVerifier [0x01689342+2502274]
	GetHandleVerifier [0x016B7959+2692249]
	GetHandleVerifier [0x016BABDC+2705180]
	GetHandleVerifier [0x014C3480+643008]
	(No symbol) [0x013B208F]
	(No symbol) [0x013B7AB8]
	(No symbol) [0x013B7BA5]
	(No symbol) [0x013C273B]
	BaseThreadInitThunk [0x

In [10]:
# length of arrays
print(len(rank),len(name),len(net_worth),len(citizenship),len(source), len(industry),len(age))

2626 2626 2626 2626 2626 2626 2626


In [11]:
# creating a dataframe
billionaire_data = pd.DataFrame({'Name':name, 'Rank':rank, 'Age':age, 'Net-Worth':net_worth, 'Citizenship':citizenship,
                                'Income_Source':source, 'Industry':industry})
billionaire_data.head()

,Name,Rank,Age,Net-Worth,Citizenship,Income_Source,Industry
0,Elon Musk,2,51,$180 B,United States,"Tesla, SpaceX",Automotive
1,Jeff Bezos,3,59,$114 B,United States,Amazon,Technology
2,Larry Ellison,4,78,$107 B,United States,Oracle,Technology
3,Warren Buffett,5,92,$106 B,United States,Berkshire Hathaway,Finance & Investments
4,Bill Gates,6,67,$104 B,United States,Microsoft,Technology


### Q9.
Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [12]:
driver.get("https://www.youtube.com/watch?v=u2NAuswnTKs")
time.sleep(5)

In [14]:
# Create empty lists
comments = []
upvotes = []
comment_time = []

# Scroll to load more comments
for _ in range(20):
    driver.execute_script("window.scrollBy(0, 1000)")
    time.sleep(5)
    
    # Extract comments
    try:
        comm = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
        for i in comm:
            comments.append(i.text)
    except NoSuchElementException:
        comments.append('-')
        
    # Extract upvotes
    try:
        up = driver.find_elements(By.XPATH, '//*[@id="vote-count-middle"]')
        for i in up:
            upvotes.append(i.text)
    except NoSuchElementException:
        upvotes.append('-')


    # Extract comment timestamps
    try:
        comm_tm = driver.find_elements(By.XPATH, '//*[@id="header-author"]/yt-formatted-string/a')
        for i in comm_tm:
            comment_time.append(i.text)
    except NoSuchElementException:
        comment_time.append('-')

    # Check if the desired number of comments is reached
    for i in range(len(comments)):
        if i>=700:
            break



In [15]:
# Print the lengths of the lists
print(len(comments), len(upvotes), len(comment_time))

3513 3585 3585


In [17]:
yt_data = pd.DataFrame({'Comment':comments[:500],'Time_Posted':comment_time[:500], 'Comment_Upvotes':upvotes[:500]})

In [18]:
yt_data.head()

,Comment,Time_Posted,Comment_Upvotes
0,"Tu mera koi na, hoke bhi kuchh laage!\n\nAudio...",6 months ago,3.6K
1,Fallen in love with this song. I so can relate...,6 months ago,3.1K
2,Bollywood is nothing without Arijit Singh ️,2 days ago,21
3,"""Tu Mera koi na hoke bhi kuch lage"" this line ...",2 months ago,395
4,Dear parents: Just because your child is smili...,4 months ago,1.9K


### Q10.
Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description

In [ ]:
driver.get("https://www.hostelworld.com/")
time.sleep(5)

In [ ]:
try:
    location = driver.find_element(By.XPATH, '//div[@class="search"]/input')
    location.send_keys('London')
except NoSuchElementException:
    print("Exception ocured")
    location = driver.find_element(By.CLASS_NAME, 'search-input')
    loation.send_keys('London')
    
select_ = driver.find_element(By.XPATH, '//div[@class="label"]')
select_.click()

go_button = driver.find_element(By.XPATH, '//div[@class="search-button"]/button')
go_button.click()

In [27]:
# creating empty lists
name = []
distance = []
ratings = []
reviews = []
private_price = []
dorm_price = []
facilities = []
desciption = []


# scraping data

try:
    dist = driver.find_elements(By.XPATH, '//div[@class="property-distance"]/span[2]')
    for i in dist:
        distance.append(i.text)
except NoSuchElementException:
    distance.append('-')
    
hostel_url = []

# scraping all hostel url
url = driver.find_elements(By.XPATH, '//div[@class="property-card"]/a')
for i in url:
    hostel_url.append(i.get_attribute('href'))
    time.sleep(3)
    
hostel_url = hostel_url[1:]

In [28]:
print(len(distance), len(hostel_url))

0 0
